In [1]:
import os
import cv2
import glob
import pandas as pd
import numpy as np
import similaritymeasures
from tqdm.notebook import tqdm
from utils.functions import checkDirExist

# get dataset trajectory (csv)
dataset_dir = os.path.join(os.getcwd(), 'dataset/position_5/trajectory-100/')
dataset_list = glob.glob(dataset_dir + "*.csv")

# get test trajectory (csv)
test_dir = os.path.join(os.getcwd(), 'dataset/evaluation/1_player/')
test_list = glob.glob(test_dir + "*.csv")

# get trajectory by lstm model(csv)
lstm_dir = os.path.join(os.getcwd(), 'dataset/evaluation/2_machine/')
lstm_list = glob.glob(lstm_dir + "*.csv")

print('dataset: ', len(dataset_list))
print('lstm model: ', len(lstm_list))
print('test: ', len(test_list))

dataset:  1162
lstm model:  20
test:  20


In [29]:


lstm_dir

'/root/notebooks/CoachAI/dataset/evaluation/2_machine/'

## Predict by Brute-force

In [30]:
def compare_trajectory(df1, df2):
    '''
        input: two trajectory dataframe
        output: min loss for trajectory
        NOTE: df1 must shorter than df2
    '''
    loss = 9999
    for times in range(len(df2)-len(df1)+1):
        euclidean_dist = 0
        for idx, _ in enumerate(df1):
            x = df1.loc[idx, 'X'] - df2.loc[idx+times, 'X']
            y = df1.loc[idx, 'Y'] - df2.loc[idx+times, 'Y']
            euclidean_dist += np.linalg.norm([x, y])
        
        if euclidean_dist < loss:
            loss = euclidean_dist
    return loss

# save best match trajectory as dataframe
df = pd.DataFrame()

for idx, test_csv in enumerate(tqdm(test_list)):  # get all test trajectory
    test_df = pd.read_csv(test_csv)
    df.loc[idx, 'Loss(bf)'] = 9999
    df.loc[idx, 'Loss(lstm)'] = 9999
    df.loc[idx, 'test'] = test_csv.split('/')[-1]
    no = test_csv.split('NUTN-')[1].split('_')[0]
    df.loc[idx, 'lstm-model'] = glob.glob(lstm_dir + "*NUTN-{}*".format(no))[0].split('/')[-1]
    
    for dataset_csv in dataset_list:  # get all dataset trajectory
        dataset_df = pd.read_csv(dataset_csv)
        if len(test_df) > len(dataset_df):  # test_df must shorter than dataset_df
            continue
        else:
            deltaX = dataset_df.loc[0, 'X'] - test_df.loc[0, 'X']
            deltaY = dataset_df.loc[0, 'Y'] - test_df.loc[0, 'Y']
            
            # compute trajectory offset value for x, y (brute-force)
            dataset_df['X'] = dataset_df['X'] - deltaX
            dataset_df['Y'] = dataset_df['Y'] - deltaY
            loss = compare_trajectory(test_df, dataset_df)
            if loss < df.loc[idx, 'Loss(bf)']:
                df.loc[idx, 'Loss(bf)'] = loss
                df.loc[idx, 'brute-force'] = dataset_csv.split('/')[-1]
                df.loc[idx, 'delta(bf)'] = str('{},{}'.format(deltaX, deltaY))
    
    # compute trajectory offset value for x, y (lstm-model)
    lstm_df = pd.read_csv(glob.glob(lstm_dir + "*NUTN-{}*".format(no))[0])
    deltaX = lstm_df.loc[0, 'X'] - test_df.loc[0, 'X']
    deltaY = lstm_df.loc[0, 'Y'] - test_df.loc[0, 'Y']
    lstm_df['X'] = lstm_df['X'] - deltaX
    lstm_df['Y'] = lstm_df['Y'] - deltaY
    loss = compare_trajectory(test_df, lstm_df)
    df.loc[idx, 'Loss(lstm)'] = loss
    df.loc[idx, 'delta(lstm)'] = str('{},{}'.format(deltaX, deltaY))
    
    test_curve = test_df.loc[:, ['X', 'Y']].values
    lstm_curve = lstm_df.loc[:, ['X', 'Y']].values
    bf_df = pd.read_csv(dataset_dir+df.loc[idx, 'brute-force'])
    bf_curve = bf_df.loc[:, ['X', 'Y']].values
    df.loc[idx, 'similarity(test-lstm)'] = similaritymeasures.frechet_dist(test_curve, lstm_curve)
    df.loc[idx, 'similarity(test-bf)'] = similaritymeasures.frechet_dist(test_curve, bf_curve)
df.sort_values("test", inplace=True)
df

,Loss(bf),Loss(lstm),test,lstm-model,brute-force,delta(bf),delta(lstm),similarity(test-lstm),similarity(test-bf)
19,27.490563,155.117729,NUTN-01_5_150_00_00_00_predict.csv,NUTN-01_5_150_40_16_31_predict.csv,NUTN_5_150_40_20_30_1_predict_100.csv,"-11,-30","-10,23",79.624117,75.643903
9,34.708204,99.169964,NUTN-02_5_150_00_00_00_predict.csv,NUTN-02_5_150_20_68_35_predict.csv,NUTN_5_150_20_70_40_1_predict_100.csv,"-4,-9","-8,64",108.226614,84.717177
0,40.363205,338.741083,NUTN-03_5_150_00_00_00_predict.csv,NUTN-03_5_150_40_55_35_predict.csv,NUTN_5_150_60_50_30_1_predict_100.csv,"-27,22","-16,77",115.663304,154.651220
8,87.948620,100.331953,NUTN-04_5_150_00_00_00_predict.csv,NUTN-04_5_150_60_10_39_predict.csv,NUTN_5_150_60_10_40_2_predict_100.csv,"11,9","22,64",53.075418,87.726849
18,30.402607,109.927736,NUTN-05_5_150_00_00_00_predict.csv,NUTN-05_5_150_70_27_33_predict.csv,NUTN_5_150_60_30_35_3_predict_100.csv,"2,3","13,58",302.676395,272.132321
15,62.236389,116.999464,NUTN-06_5_150_00_00_00_predict.csv,NUTN-06_5_150_30_10_40_predict.csv,NUTN_5_150_80_20_30_2_predict_100.csv,"-6,-43","0,0",127.098387,122.016392
17,27.214067,119.889967,NUTN-07_5_150_00_00_00_predict.csv,NUTN-07_5_150_40_68_38_predict.csv,NUTN_5_150_60_60_35_2_predict_100.csv,"-27,-27","-6,7",342.811902,229.490741
16,40.653267,116.390386,NUTN-08_5_150_00_00_00_predict.csv,NUTN-08_5_150_40_18_40_predict.csv,NUTN_5_150_60_30_35_3_predict_100.csv,"-4,-34","7,21",99.005050,88.526832
10,42.415213,93.242959,NUTN-09_5_150_00_00_00_predict.csv,NUTN-09_5_150_26_23_36_predict.csv,NUTN_5_150_60_30_30_1_predict_100.csv,"-15,-75","0,13",118.152444,76.485293
3,47.533305,137.361040,NUTN-10_5_150_00_00_00_predict.csv,NUTN-10_5_150_38_68_37_predict.csv,NUTN_5_150_40_60_35_2_predict_100.csv,"-10,-102","-10,10",113.771701,113.437207


In [31]:
# Save result to csv
result_dir = os.path.join(os.getcwd(), 'predictions/')
checkDirExist(result_dir)
df.to_csv(os.path.join(result_dir, 'evaluation_result.csv'), index=False)

## Overlay

In [12]:
def plot(img, df, point_color):
    for index in range(len(df['X'])):
        y, x = df.loc[index, 'X'], df.loc[index, 'Y']
        
        # draw point as circle
        for i in range(-1, 2):
            for j in range(-1, 2):
                try:
                    img[x+i, y+j] = point_color
                except IndexError as e:
                    pass
    return img
    

def main():
    for idx in tqdm(df.index):
        # Read csv file
        test = pd.read_csv(test_dir+df.loc[idx, 'test'])
        lstm = pd.read_csv(lstm_dir+df.loc[idx, 'lstm-model'])
        dataset = pd.read_csv(dataset_dir+df.loc[idx, 'brute-force'])
        
        lstm['X'] -= int(df.loc[idx, 'delta(lstm)'].split(',')[0])
        lstm['Y'] -= int(df.loc[idx, 'delta(lstm)'].split(',')[1])
        dataset['X'] -= int(df.loc[idx, 'delta(bf)'].split(',')[0])
        dataset['Y'] -= int(df.loc[idx, 'delta(bf)'].split(',')[1])
        
        # Plot
        img = np.ones((1080, 1920, 3), np.uint8)*255
        plot(img, test, [0, 0, 255])
        plot(img, lstm, [0, 255, 0])
        plot(img, dataset, [255, 0, 0])
        
        # Save as image
        cv2.imwrite('predictions/evaluation_%02d.jpg'%(idx+1), img)

if __name__ == "__main__":
    main()

In [39]:
import random
def plot(img, df, point_color):
    for index in range(len(df['X'])):
        y, x = df.loc[index, 'X'], df.loc[index, 'Y']
        
        # draw point as circle
        for i in range(-1, 2):
            for j in range(-1, 2):
                try:
                    img[x+i, y+j] = point_color
                except IndexError as e:
                    pass
    return img
    

csvs = glob.glob('compare/*')
img = np.ones((1080, 1920, 3), np.uint8)*255
for csv in csvs:
    df = pd.read_csv(csv)
    plot(img, df, [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)])
    cv2.imwrite('compare/overlay.jpg', img)

In [72]:
df = pd.DataFrame()
csvs = glob.glob('compare/*.csv')
csvs.sort()
for idx, csv in enumerate(csvs):
    base = csv
    compare = csvs.copy()
    compare.remove(base)
    
    loss = []
    for i, csv in enumerate(compare):
        base_df = pd.read_csv(base)
        compare_df = pd.read_csv(csv)
        base_curve = base_df.loc[:, ['X', 'Y']].values
        compare_curve = compare_df.loc[:, ['X', 'Y']].values
        df.loc[idx, 'similarity{}'.format(i)] = similaritymeasures.frechet_dist(base_curve, compare_curve)
        loss.append(similaritymeasures.frechet_dist(base_curve, compare_curve))
    df.loc[idx, 'mean sim'] = np.mean(loss)
df

,similarity0,similarity1,similarity2,similarity3,similarity4,similarity5,similarity6,similarity7,similarity8,mean sim
0,70.604532,44.777226,25.612497,67.801180,62.433965,49.203658,33.241540,21.400935,38.327536,45.933674
1,70.604532,39.661064,81.394103,22.090722,109.457754,36.878178,81.252692,63.253458,68.154237,63.638527
2,44.777226,39.661064,50.447993,35.735137,78.447435,11.180340,47.434165,39.000000,35.735137,42.490944
3,25.612497,81.394103,50.447993,78.721026,44.407207,56.859476,25.495098,33.837849,32.280025,47.672808
4,67.801180,22.090722,35.735137,78.721026,107.298649,42.544095,78.854296,60.440053,64.513564,61.999858
5,62.433965,109.457754,78.447435,44.407207,107.298649,83.216585,33.837849,67.082039,46.043458,70.247216
6,49.203658,36.878178,11.180340,56.859476,42.544095,83.216585,55.081757,42.720019,40.112342,46.421828
7,33.241540,81.252692,47.434165,25.495098,78.854296,33.837849,55.081757,36.619667,18.384776,45.577982
8,21.400935,63.253458,39.000000,33.837849,60.440053,67.082039,42.720019,36.619667,38.013156,44.707464
9,38.327536,68.154237,35.735137,32.280025,64.513564,46.043458,40.112342,18.384776,38.013156,42.396026


In [51]:
df['mean sim'].min()

42.39602561769224

In [70]:
df = pd.DataFrame()
csvs = glob.glob('compare/*.csv')
csvs.sort()
for idx, csv in enumerate(csvs):
    base = csv
    compare = csvs.copy()
    compare.remove(base)
    df.loc[idx, 'Loss'] = 9999
    
    loss = []
    for i, csv in enumerate(compare):
        base_df = pd.read_csv(base)
        compare_df = pd.read_csv(csv)
        if len(base_df) >= len(compare_df):  # test_df must shorter than dataset_df
            loss = compare_trajectory(base_df, compare_df)
        else:
            loss = compare_trajectory(compare_df, base_df)
        
        if loss < df.loc[idx, 'Loss']:
            df.loc[idx, 'Loss'] = loss
df

,Loss
0,52.499072
1,95.602707
2,42.322590
3,49.252921
4,9999.000000
5,93.717050
6,42.322590
7,49.252921
8,121.668200
9,74.283815


In [71]:
df = df.drop(4)
df['Loss'].mean()

68.9913184542937